In [78]:
import numpy as np
import pandas as pd
from constants import *
import matplotlib.pyplot as plt

sb_data_file = 'D:/UVic/CSC503Proj/data/merged_data.csv'
sb_filtered_data_file = 'D:/UVic/CSC503Proj/data/filtered_data.csv'

unwanted_cols__ = ['Unnamed: 0', 'videoID', 'UUID', 'description', 'title', 'userID', 'channelID', 'incorrectVotes']

In [79]:
# Read the file, filter the columns, and save the new file


# df = pd.read_csv(sb_data_file, low_memory=False)
# filtered_df = df.loc[:, ~df.columns.isin(unwanted_cols__)]
# filtered_df.to_csv(sb_filtered_data_file, index=False)

# print(f"Filtered data has been written to {sb_filtered_data_file}")

In [80]:
sb_data = pd.read_csv(sb_filtered_data_file, 
                      dtype=dtypes,
                      usecols=lambda col: col not in unwanted_cols__,
                      #converters={'timeSubmitted':convert_timestamp_to_seconds},
                      # skiprows=range(1, 6_000_000),
                      # nrows = 8_000_000,
                      na_values={'videoDuration': [0],
                                 'published': [0]})

pd.set_option('display.max_columns', None)
display(sb_data.head())

,startTime,endTime,votes,locked,timeSubmitted,views,category,actionType,videoDuration,hidden,reputation,shadowHidden,published,isVIP
0,714.00000,763.00000,64,False,1.564089e+09,386,sponsor,skip,NaN,0,0.0,False,1.529539e+09,False
1,446.51013,513.39233,225,False,1.564089e+09,3222,sponsor,skip,NaN,1,0.0,False,1.562458e+09,False
2,488.52150,542.11035,-2,False,1.564089e+09,25661,sponsor,skip,552.0,0,0.0,False,1.534032e+09,False
3,497.14310,569.15330,-2,False,1.564089e+09,24217,sponsor,skip,NaN,0,0.0,False,1.517443e+09,False
4,41.00000,53.00000,115,False,1.564089e+09,768,sponsor,skip,NaN,0,0.0,False,1.529539e+09,False


In [81]:
# readCol = ['category']
# sb_data_cat = pd.read_csv(sb_filtered_data_file, 
#                       dtype=dtypes,
#                       usecols=lambda col: col in readCol)
# print(len(sb_data_cat))
# sb_data_cat['category'].value_counts()
# print(len(sb_data_cat))

In [82]:
print(len(sb_data))

15334922


In [83]:
sb_data = sb_data[sb_data['category'] != 'moreCategories']

In [84]:
print(len(sb_data))

15334918


In [92]:
sb_data['category'].cat.remove_unused_categories()
sb_data['category'].value_counts()

category
sponsor             3483201
intro               2742959
filler              2676918
outro               2107166
selfpromo           1339473
interaction         1177598
poi_highlight        771380
preview              421595
music_offtopic       404021
chapter              190634
exclusive_access      19973
moreCategories            0
Name: count, dtype: int64

In [93]:
fraction = 0.2

stratified_sample = sb_data.groupby('category', group_keys=False).apply(lambda x: x.sample(frac=fraction))
stratified_sample.reset_index(drop=True, inplace=True)

C:\Users\joyji\AppData\Local\Temp\ipykernel_12748\6632669.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  stratified_sample = sb_data.groupby('category', group_keys=False).apply(lambda x: x.sample(frac=fraction))
C:\Users\joyji\AppData\Local\Temp\ipykernel_12748\6632669.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = sb_data.groupby('category', group_keys=False).apply(lambda x: x.sample(frac=fraction))


In [94]:
print(len(stratified_sample))

3066985


In [95]:
stratified_sample['category'].value_counts()

category
sponsor             696640
intro               548592
filler              535384
outro               421433
selfpromo           267895
interaction         235520
poi_highlight       154276
preview              84319
music_offtopic       80804
chapter              38127
exclusive_access      3995
moreCategories           0
Name: count, dtype: int64

In [ ]:
# stratified_sample['segmentDuration'] = stratified_sample['endTime']-stratified_sample['startTime']
# stratified_sample['scaledStart'] = stratified_sample['startTime']/stratified_sample['videoDuration']
# stratified_sample['scaledEnd'] = stratified_sample['endTime']/stratified_sample['videoDuration']
# stratified_sample['scaledSegmentDuration'] = stratified_sample['segmentDuration']/stratified_sample['videoDuration']
# stratified_sample['timeToSubmission'] = stratified_sample['timeSubmitted']-stratified_sample['published']

In [103]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [105]:
train_set, test_set = train_test_split(stratified_sample, test_size=0.2, random_state=42)
len(train_set), len(test_set)


sb = train_set.drop("category", axis=1)
sb_labels = train_set["category"].copy()
sb_test = test_set.drop("category", axis=1)
sb_labels_test = test_set["category"].copy()

sb.head()
sb_labels.head()


1775487        outro
2116207    selfpromo
875103         intro
502825        filler
2824427      sponsor
Name: category, dtype: category
Categories (12, object): ['filler', 'interaction', 'intro', 'music_offtopic', ..., 'moreCategories', 'poi_highlight', 'exclusive_access', 'chapter']

In [ ]:
num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                            StandardScaler())

cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"),
                                        OneHotEncoder(handle_unknown='ignore'))

num_attribs = ['startTime', 'endTime', 'votes', 'timeSubmitted', 'views', 'videoDuration', 'reputation', 'published']
cat_attribs = ['locked', 'category', 'actionType', 'hidden', 'shadowHidden', 'isVIP']


preprocess_pipeline = ColumnTransformer(
[('num', num_pipeline, num_attribs),
('cat', cat_pipeline, cat_attribs)])


stratified_sample_processed = preprocess_pipeline.fit_transform(stratified_sample)